In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

# plt.style.use('dark_background')
random_state=42

In [2]:
tesis = pd.read_csv("../Datos/tesis_final_preprocesado.csv")
tesis.head()

,idPaciente,Mes,TAS,Adherencia,Sexo,Edad,tas_basal,Adherencia_Acumulada,Adherencia_Total,Adherencia_Perfecta,...,Intercept,Adherencia_lag1,TAS_lag1,Adherencia_Acumulada_lag2,TAS_Media_Acumulada_lag2,Mes*Adherencia,Mes*Adherencia_Perfecta,Mes*Adherencia_Total,Mes*Adherencia_lag1,Mes*Adherencia_Acumulada
0,4026,1,119.000000,1,0,76.0,116,1.0,1.0,1,...,1,0.0,116.00000,0.0,116.000000,1,1,1.0,0.0,1.0
1,4026,2,127.000000,1,0,76.0,116,1.0,1.0,1,...,1,1.0,119.00000,0.0,116.000000,2,2,2.0,2.0,2.0
2,4026,3,140.000000,1,0,76.0,116,1.0,1.0,1,...,1,1.0,127.00000,1.0,119.000000,3,3,3.0,3.0,3.0
3,4026,4,146.712710,1,0,76.0,116,1.0,1.0,1,...,1,1.0,140.00000,1.0,123.000000,4,4,4.0,4.0,4.0
4,4026,5,177.708084,1,0,76.0,116,1.0,1.0,1,...,1,1.0,146.71271,1.0,128.666667,5,5,5.0,5.0,5.0


### Modelo base

In [3]:
fixed_effects = [
    "Intercept",
    "Sexo",
    "Edad",
    "Mes",
]

In [25]:
mixed = sm.MixedLM(
    endog=tesis["TAS"],
    exog=tesis[["Intercept", "Sexo", "Edad", "Adherencia", "Adherencia_Total", "Mes", "Mes*Adherencia", "Mes*Adherencia_Total"]],
    groups=tesis["idPaciente"],
    exog_re=tesis[["Intercept", "Mes"]]
).fit(reml=False)
print(mixed.summary())
round(mixed.aic, 2), round(mixed.bic, 2)

              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    TAS        
No. Observations:    3920       Method:                ML         
No. Groups:          560        Scale:                 113.2189   
Min. group size:     7          Log-Likelihood:        -15389.8304
Max. group size:     7          Converged:             Yes        
Mean group size:     7.0                                          
------------------------------------------------------------------
                      Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
------------------------------------------------------------------
Intercept            121.987    3.032 40.235 0.000 116.044 127.929
Sexo                   3.723    0.747  4.983 0.000   2.259   5.187
Edad                   0.171    0.039  4.429 0.000   0.095   0.246
Adherencia            -2.055    1.325 -1.550 0.121  -4.652   0.543
Adherencia_Total       0.662    2.784  0.238 0.812  -4.795   6.119
Mes       

(30803.66, 30878.95)

In [22]:
mixed = sm.MixedLM(
    endog=tesis["TAS"],
    exog=tesis[["Intercept", "Sexo", "Edad", "Adherencia_Total", "Mes", "Mes*Adherencia"]],
    groups=tesis["idPaciente"],
    exog_re=tesis[["Intercept", "Mes"]]
).fit(reml=False)
print(mixed.summary())
round(mixed.aic, 2), round(mixed.bic, 2)

              Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   TAS        
No. Observations:     3920      Method:               ML         
No. Groups:           560       Scale:                113.3521   
Min. group size:      7         Log-Likelihood:       -15391.0643
Max. group size:      7         Converged:            Yes        
Mean group size:      7.0                                        
-----------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------
Intercept           122.384    2.637 46.413 0.000 117.215 127.552
Sexo                  3.718    0.747  4.975 0.000   2.254   5.183
Edad                  0.170    0.039  4.409 0.000   0.094   0.245
Adherencia_Total     -1.774    1.678 -1.057 0.290  -5.062   1.514
Mes                   0.574    0.158  3.634 0.000   0.265   0.884
Mes*Adherencia       -1.

(30802.13, 30864.87)